In [1]:
# !pip3 install --ignore-installed PyYAML -U
# # make sure pydantic is 1.10.8
# !pip3 install pydantic=='1.10.8' -U
# !pip3 install langchain pandas openai -U
# !pip3 install python-dotenv
# !pip3 install langchain --upgrade -U

In [2]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI
import pandas as pd
import os
import openai 
import sys
# sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key  = os.environ['OPENAI_API_KEY']
openai.api_key= os.environ['OPENAI_API_KEY']

print('lib and key are loaded')

lib and key are loaded


In [3]:
df = pd.read_csv("fake port.csv")
df

,Stock,Sector,Weight,Return
0,SPY,FINANCE,3.08,124.1%
1,MCK,HEALTH CARE,0.61,85.9%
2,LLY,HEALTH CARE,0.54,68.5%
3,META,COMMUNICATIONS,1.30,61.8%
4,HCA,HEALTH CARE,0.55,52.6%
5,CMG,CONSUMER DISCRETIONARY,0.73,50.9%
6,HSY,CONSUMER STAPLES,0.47,50.6%
7,MRK,HEALTH CARE,0.59,38.9%
8,IAU,MATERIALS,0.42,36.8%
9,BSX,HEALTH CARE,0.59,32.1%


In [4]:
# agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True)

In [5]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    # ChatOpenAI(temperature=0, model="gpt-4"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

agent.run("how many rows are there?")



> Entering new  chain...



Invoking: `python_repl_ast` with `df.shape[0]`


50There are 50 rows in the dataframe.

> Finished chain.


'There are 50 rows in the dataframe.'

In [6]:
# One suggested thread to fix the JSONDecodeError issue: https://github.com/hwchase17/langchain/issues/6364

### Prompt0: use simple prompts. Simple prompt has the OutputParserException: Could not parse tool input: {'name': 'python', 'arguments': <python_code>}


In [7]:
agent.run("What are the worst performers based returns?")



> Entering new  chain...


OutputParserException: Could not parse tool input: {'name': 'python', 'arguments': "worst_performers = df.sort_values('Return').head()\nworst_performers"} because the `arguments` is not valid JSON.

### Prompt1: worst performers are just reverse of top performers

In [ ]:
agent.run("""You are an equity analyst. This dataframe is a portfolio's holdings of stocks. Stock column is the stock tickers.
          Sector column is each stock's sector. Weight column is stock weights in the portfolio. Return column is stock performance. 
          The table is sorted by Return column in descending order. 
          Using all data from this portfolio, please comment the top and worst performers in table format.
          """)




> Entering new  chain...
To comment on the top and worst performers in the portfolio, we can look at the stocks with the highest and lowest returns, respectively. 

The top performers in the portfolio are:

| Stock | Sector         | Weight | Return |
|-------|----------------|--------|--------|
| SPY   | FINANCE        | 3.08   | 124.1% |
| MCK   | HEALTH CARE    | 0.61   | 85.9%  |
| LLY   | HEALTH CARE    | 0.54   | 68.5%  |
| META  | COMMUNICATIONS | 1.3    | 61.8%  |
| HCA   | HEALTH CARE    | 0.55   | 52.6%  |

The worst performers in the portfolio are:

| Stock | Sector         | Weight | Return |
|-------|----------------|--------|--------|
| HCA   | HEALTH CARE    | 0.55   | 52.6%  |
| META  | COMMUNICATIONS | 1.3    | 61.8%  |
| LLY   | HEALTH CARE    | 0.54   | 68.5%  |
| MCK   | HEALTH CARE    | 0.61   | 85.9%  |
| SPY   | FINANCE        | 3.08   | 124.1% |

Please note that the worst performers are based on the lowest returns in the portfolio.

> Finished chain.


'To comment on the top and worst performers in the portfolio, we can look at the stocks with the highest and lowest returns, respectively. \n\nThe top performers in the portfolio are:\n\n| Stock | Sector         | Weight | Return |\n|-------|----------------|--------|--------|\n| SPY   | FINANCE        | 3.08   | 124.1% |\n| MCK   | HEALTH CARE    | 0.61   | 85.9%  |\n| LLY   | HEALTH CARE    | 0.54   | 68.5%  |\n| META  | COMMUNICATIONS | 1.3    | 61.8%  |\n| HCA   | HEALTH CARE    | 0.55   | 52.6%  |\n\nThe worst performers in the portfolio are:\n\n| Stock | Sector         | Weight | Return |\n|-------|----------------|--------|--------|\n| HCA   | HEALTH CARE    | 0.55   | 52.6%  |\n| META  | COMMUNICATIONS | 1.3    | 61.8%  |\n| LLY   | HEALTH CARE    | 0.54   | 68.5%  |\n| MCK   | HEALTH CARE    | 0.61   | 85.9%  |\n| SPY   | FINANCE        | 3.08   | 124.1% |\n\nPlease note that the worst performers are based on the lowest returns in the portfolio.'

### Prompt2: To find top holdings. Results are incorrect. Not actually sorting.

In [ ]:
agent.run("Using all data from this portfolio, please comment the top holdings with the highest weights in the portfolio in table format. ")



> Entering new  chain...
To identify the top holdings with the highest weights in the portfolio, we can sort the dataframe `df` by the `Weight` column in descending order and select the top holdings. Here is the table format of the top holdings:

| Stock | Sector         | Weight | Return |
|-------|----------------|--------|--------|
| SPY   | FINANCE        | 3.08   | 124.1% |
| META  | COMMUNICATIONS | 1.3    | 61.8%  |
| MCK   | HEALTH CARE    | 0.61   | 85.9%  |
| LLY   | HEALTH CARE    | 0.54   | 68.5%  |
| HCA   | HEALTH CARE    | 0.55   | 52.6%  |

Please note that the table is sorted by the `Weight` column in descending order.

> Finished chain.


'To identify the top holdings with the highest weights in the portfolio, we can sort the dataframe `df` by the `Weight` column in descending order and select the top holdings. Here is the table format of the top holdings:\n\n| Stock | Sector         | Weight | Return |\n|-------|----------------|--------|--------|\n| SPY   | FINANCE        | 3.08   | 124.1% |\n| META  | COMMUNICATIONS | 1.3    | 61.8%  |\n| MCK   | HEALTH CARE    | 0.61   | 85.9%  |\n| LLY   | HEALTH CARE    | 0.54   | 68.5%  |\n| HCA   | HEALTH CARE    | 0.55   | 52.6%  |\n\nPlease note that the table is sorted by the `Weight` column in descending order.'

### Prompt2.1: further prompt to ask it sort by Weight column. Encountered the JSONDecodeError again

In [ ]:
agent.run("""Using all data from this portfolio, please sort the dataframe by 'Weight' column and find top holdings with the highest weights in the portfolio in table format. """)



> Entering new  chain...


OutputParserException: Could not parse tool input: {'name': 'python', 'arguments': "df_sorted = df.sort_values(by='Weight', ascending=False)\ntop_holdings = df_sorted.head()\ntop_holdings"} because the `arguments` is not valid JSON.

### Trying solve the OutputParserException with prompt - failed

In [ ]:
agent.run("""Using all data from this portfolio, please sort the dataframe by 'Weight' column and find top holdings with the highest weights in the portfolio in table format.
          When generating tool input in json format, replace 'name': 'python' with 'name': 'python_repl_ast' if any.
          """)



> Entering new  chain...


OutputParserException: Could not parse tool input: {'name': 'python', 'arguments': "import pandas as pd\n\n# Sort the dataframe by 'Weight' column in descending order\nsorted_df = df.sort_values(by='Weight', ascending=False)\n\n# Get the top holdings with the highest weights\ntop_holdings = sorted_df.head()\n\ntop_holdings"} because the `arguments` is not valid JSON.

In [ ]:
agent.run("""You are an equity analyst. This dataframe is a portfolio's holdings of stocks. 
          Sector column is each security's sector. Weight column is stock weights in the portfolio. Return column is stock performance. 
          The table is sorted by Return column in descending order. 
          Using all data from this portfolio, please show the worst performers with lowest returns in table format.
          """)




> Entering new  chain...


OutputParserException: Could not parse tool input: {'name': 'python', 'arguments': "df_sorted = df.sort_values('Return', ascending=False)\nbest_performers = df_sorted.head()\nbest_performers"} because the `arguments` is not valid JSON.

### Attempt 2 combine the queries - chatgpt only show python code, not invoking python REPL to ge

In [ ]:
# tabulate top performers with largest Return, and tablue worst performers with least Return. 
#           Aggregate securities by sector and show the sectors' aggregated "% of Portfolio" in table format.

### Prompt3: same OutputParserException error 

In [ ]:
agent.run("""You are an equity analyst. This data is a portfolio's holdings. 
          Sector column is each security's sector. 
          Weight column is security weights. Return column is security performance. 
          The table is sorted by Return column in descening order. 
          Please show the portfolio's best performers.
          """)




> Entering new  chain...


OutputParserException: Could not parse tool input: {'name': 'python', 'arguments': 'best_performers = df.head()\nbest_performers'} because the `arguments` is not valid JSON.

### Prompt 4: split the request

In [ ]:
agent.run("""You are an equity analyst. This data is a portfolio's stock holdings. Sector column is each stock's sector. 
          Weight column is stock weights in the portfolio. Return column is security performance. 
          The table is sorted by Return column in descening order. 
          Please show the portfolio's top holdings with largest Weight in table format.
          """)




> Entering new  chain...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


OutputParserException: Could not parse tool input: {'name': 'python', 'arguments': "df_sorted = df.sort_values('Weight', ascending=False)\ntop_holdings = df_sorted.head()\ntop_holdings"} because the `arguments` is not valid JSON.

In [ ]:
# , tabulate top performers with largest Return, and tablue worst performers with least Return. 
#           Aggregate securities by sector and show the sectors' aggregated "% of Portfolio" in table format.